# Testing ResNet

In [83]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [84]:
# load CIFAR100 dataset
transform = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])
cifar100_train = datasets.CIFAR100(root='./data_cifar100', train=True, download=True, transform=transform)
cifar100_test = datasets.CIFAR100(root='./data_cifar100', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [85]:
# initialize data loader
BATCH_SIZE = 128
train_loader = DataLoader(cifar100_train, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(cifar100_test, batch_size = BATCH_SIZE, shuffle = False)

In [86]:
cifar100_test.data[0, :, :].shape

(32, 32, 3)

In [87]:
# set GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [88]:
# load ResNet50 model
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained = True)

Using cache found in C:\Users\fongc/.cache\torch\hub\pytorch_vision_v0.6.0


In [89]:
# do transfer learning on model
model.fc = torch.nn.Linear(2048, 100)
model = model.to(device)

In [90]:
# freeze all layers except the last one
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

In [91]:
# define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr = 0.001, momentum = 0.9)

In [92]:
# train model
EPOCHS = 40
for epoch in range(EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,   100] loss: 4.576
[1,   200] loss: 4.365
[1,   300] loss: 4.201
[2,   100] loss: 3.963
[2,   200] loss: 3.879
[2,   300] loss: 3.826
[3,   100] loss: 3.715
[3,   200] loss: 3.655
[3,   300] loss: 3.618
[4,   100] loss: 3.574
[4,   200] loss: 3.539
[4,   300] loss: 3.497
[5,   100] loss: 3.441
[5,   200] loss: 3.451
[5,   300] loss: 3.447


In [ ]:
# make predictions
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [ ]:
print("Accuracy:", correct / total)

Accuracy: 0.2656
